<a href="https://colab.research.google.com/github/andrejdaskalov/rec-sys-evaluation-paper/blob/main/PaperLLMRecommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
DATASET_DIR = "/gdrive/MyDrive/movielens_small/ml-latest-small/"
def get_dataset_path(filename: str) -> str:
    return DATASET_DIR + filename

this function is used to standardize model run names and path

In [ ]:
import datetime
RESULT_PATH = "results/"
def get_results_path(model_name: str, dataset_type: str) -> str:
    current_time = datetime.datetime.now()
    datestr = "T".join(str(current_time).split(" "))
    subdir = RESULT_PATH + "_".join([model_name, dataset_type, datestr])
    return get_dataset_path(subdir + "/")

# Load and import

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import os

In [ ]:
movie_data = pd.read_csv(get_dataset_path("enriched.csv"), na_filter=False)
movie_data.head()

,movieId,title,genres,tag,imdbId,cast,writers,producers,runtime,boxOfficeBudget,plotOutline,year
0,1,Toy Story,adventure|animation|children|comedy|fantasy,pixar|pixar|fun,114709,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter|Pete Docter|Andrew Stanton|Joe R...,Bonnie Arnold|Ed Catmull|Ralph Guggenheim|Stev...,81.0,30000000.0,A little boy named Andy loves to be in his roo...,1995
1,2,Jumanji,adventure|children|fantasy,fantasy|magic board game|Robin Williams|game,113497,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh|Greg Taylor|Jim Strain|Greg...,Robert W. Cort|Ted Field|Larry Franco|Scott Kr...,104.0,50000000.0,"Jumanji, one of the most unique--and dangerous...",1995
2,3,Grumpier Old Men,comedy|romance,moldy|old,113228,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson|Mark Steven Johnson,Richard C. Berman|John Davis|George Folsey Jr....,101.0,25000000.0,Things don't seem to change much in Wabasha Co...,1995
3,4,Waiting to Exhale,comedy|drama|romance,N/A,114885,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan|Terry McMillan|Ron Bass,Ron Bass|Caron K|Terry McMillan|Deborah Schind...,124.0,16000000.0,This story based on the best selling novel by ...,1995
4,5,Father of the Bride Part II,comedy,pregnancy|remake,113041,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett|Frances Goodrich|Nancy Meyers|C...,Carol Baum|Bruce A. Block|Julie B. Crane|Jim C...,106.0,30000000.0,"In this sequel to ""Father of the Bride"", Georg...",1995


In [ ]:
USER_COLUMN_NAME = 'userId'
CONTENT_COLUMN_NAME = 'movieId'
RATING_COLUMN_NAME = 'rating'

In [ ]:
movie_data['genres'] = movie_data['genres'].apply(lambda x: x.split("|"))
movie_data['tag'] = movie_data['tag'].apply(lambda x: x.split("|"))
movie_data['cast'] = movie_data['cast'].apply(lambda x: x.split("|"))
movie_data['writers'] = movie_data['writers'].apply(lambda x: x.split("|"))
movie_data['producers'] = movie_data['producers'].apply(lambda x: x.split("|"))
movie_data.head()

,movieId,title,genres,tag,imdbId,cast,writers,producers,runtime,boxOfficeBudget,plotOutline,year
0,1,Toy Story,"[adventure, animation, children, comedy, fantasy]","[pixar, pixar, fun]",114709,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Pete Docter, Andrew Stanton, J...","[Bonnie Arnold, Ed Catmull, Ralph Guggenheim, ...",81.0,30000000.0,A little boy named Andy loves to be in his roo...,1995
1,2,Jumanji,"[adventure, children, fantasy]","[fantasy, magic board game, Robin Williams, game]",113497,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Jonathan Hensleigh, Greg Taylor, Jim Strain, ...","[Robert W. Cort, Ted Field, Larry Franco, Scot...",104.0,50000000.0,"Jumanji, one of the most unique--and dangerous...",1995
2,3,Grumpier Old Men,"[comedy, romance]","[moldy, old]",113228,"[Walter Matthau, Jack Lemmon, Sophia Loren, An...","[Mark Steven Johnson, Mark Steven Johnson]","[Richard C. Berman, John Davis, George Folsey ...",101.0,25000000.0,Things don't seem to change much in Wabasha Co...,1995
3,4,Waiting to Exhale,"[comedy, drama, romance]",[N/A],114885,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Terry McMillan, Terry McMillan, Ron Bass]","[Ron Bass, Caron K, Terry McMillan, Deborah Sc...",124.0,16000000.0,This story based on the best selling novel by ...,1995
4,5,Father of the Bride Part II,[comedy],"[pregnancy, remake]",113041,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Albert Hackett, Frances Goodrich, Nancy Meyer...","[Carol Baum, Bruce A. Block, Julie B. Crane, J...",106.0,30000000.0,"In this sequel to ""Father of the Bride"", Georg...",1995


In [ ]:
movie_data_indexed = movie_data.set_index(CONTENT_COLUMN_NAME)

# Datasets
> NOTE: use only one of below dataset preparation groups, otherwise they overwrite eachother

# Prepare Normal Interactions dataset

In [ ]:
DATASET_TYPE = "full"

In [ ]:
movie_ratings = pd.read_csv(get_dataset_path("ratings.csv"))
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
len(movie_ratings)

100836

split by timestamp

In [ ]:
INTERACTIONS_TEST_SIZE = 0.2
from math import ceil, floor
movie_ratings.sort_values('timestamp', inplace=True)
interactions_train_df = movie_ratings.groupby('userId', group_keys=False).apply(lambda x: x.head(ceil((1-INTERACTIONS_TEST_SIZE)*len(x))))
interactions_test_df = movie_ratings.groupby('userId', group_keys=False).apply(lambda x: x.tail(floor(INTERACTIONS_TEST_SIZE*len(x))))
interactions_train_df.head()

,userId,movieId,rating,timestamp
43,1,804,4.0,964980499
73,1,1210,5.0,964980499
183,1,2826,4.0,964980523
120,1,2018,5.0,964980523
171,1,2628,4.0,964980523


In [ ]:
interactions_train_df_indexed = interactions_train_df.set_index('userId')
interactions_test_df_indexed = interactions_test_df.set_index('userId')
movie_ratings_indexed = movie_ratings.set_index('userId')

# Prepare Reduced interactions dataset

In [ ]:
DATASET_TYPE = "reduced"

In [ ]:
interactions_train_df_indexed = pd.read_csv(get_dataset_path("reduced_interactions.csv"), index_col="userId")
interactions_test_df_indexed = pd.read_csv(get_dataset_path("reduced_interactions_heldout.csv"), index_col="userId")
movie_ratings_indexed = pd.concat([interactions_train_df_indexed, interactions_test_df_indexed])
movie_ratings_indexed.head()

In [ ]:
interactions_train_df = interactions_train_df_indexed.reset_index()
interactions_test_df = interactions_test_df_indexed.reset_index()
movie_ratings = movie_ratings_indexed.reset_index()

# Prepare Mixed Interactions dataset

In [ ]:
DATASET_TYPE = "mixed"

In [ ]:
interactions_train_df_indexed = pd.read_csv(get_dataset_path("mixed_interactions.csv"), index_col="userId")
interactions_test_df_indexed = pd.read_csv(get_dataset_path("mixed_interactions_heldout.csv"), index_col="userId")
movie_ratings_indexed = pd.concat([interactions_train_df_indexed, interactions_test_df_indexed])
movie_ratings_indexed.head()

,movieId,rating,timestamp
userId,,,
448,69640,3.0,1289145810
352,90866,4.5,1493674691
599,1623,2.5,1498516912
474,2583,4.0,1081177421
465,2278,4.0,959896203


In [ ]:
interactions_train_df = interactions_train_df_indexed.reset_index()
interactions_test_df = interactions_test_df_indexed.reset_index()
movie_ratings = movie_ratings_indexed.reset_index()

# Evaluation

In [ ]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id][CONTENT_COLUMN_NAME]
    # interacted_items = interactions_df
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [ ]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:

    def __init__(self, interactions_df_full, interactions_df_train, interactions_df_test, content_df):
        self.interactions_df_full = interactions_df_full
        self.interactions_df_train = interactions_df_train
        self.interactions_df_test = interactions_df_test
        self.content_df = content_df


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, self.interactions_df_full)
        all_items = set(self.interactions_df_full[CONTENT_COLUMN_NAME])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(list(non_interacted_items), sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index


    def _calculate_recall(self, person_interacted_items_testset, person_recs_df, person_id):
        hits_at_5_count = 0
        hits_at_10_count = 0

        interacted_items_count_testset = len(person_interacted_items_testset)
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id,
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df[CONTENT_COLUMN_NAME].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df[CONTENT_COLUMN_NAME].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items,
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        recall_metrics = {'hits@5_count':hits_at_5_count,
                          'hits@10_count':hits_at_10_count,
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return recall_metrics


    def _calculate_precision(self, person_interacted_items_testset, person_recs_df : pd.DataFrame, person_id):

        hits_at_5_count = 0
        hits_at_10_count = 0


        top_5 = person_recs_df.head(5)
        top_10 = person_recs_df.head(10)

        top_5_filtered = top_5[top_5[CONTENT_COLUMN_NAME].isin(person_interacted_items_testset)]
        top_10_filtered = top_10[top_10[CONTENT_COLUMN_NAME].isin(person_interacted_items_testset)]

        hits_at_5_count = top_5_filtered.__len__()
        hits_at_10_count = top_10_filtered.__len__()

        precision_at_5 = hits_at_5_count / 5.0
        precision_at_10 = hits_at_10_count / 10.0


        precision_metrics = {'hits@5_count':hits_at_5_count,
                          'hits@10_count':hits_at_10_count,
                          'precision@5': precision_at_5,
                          'precision@10': precision_at_10}
        return precision_metrics


    # calculate the mean reciprocal rank (MRR)
    # Mean Reciprocal Rank (MRR) at K evaluates how quickly a ranking system can show the first relevant item in the top-K results.
    # MRR = 1/U sum_(u=1, U)(1/ rank_i)
    # where U is the total number of users and i is the position of the first relevant item for user u in top K results
    # in this method, I calculate the reciprocal rank for each user to later calculate the MRR for all users
    def _calculate_rr_user(self, person_interacted_items_testset: set[int], person_recs_df: pd.DataFrame, person_id: int):


        first_relevant_item = person_recs_df[person_recs_df[CONTENT_COLUMN_NAME]\
         .isin(person_interacted_items_testset)].head(1)

        if first_relevant_item.empty:
            return {"rr": -1 }

        idx_first_relevant_item = first_relevant_item.index[0]

        rr =  1/ int( idx_first_relevant_item + 1 )
        return {
            "rr": rr
        }



    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = self.interactions_df_test.loc[person_id]
        if type(interacted_values_testset[CONTENT_COLUMN_NAME]) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset[CONTENT_COLUMN_NAME])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset[CONTENT_COLUMN_NAME])])
        # interacted_items_count_testset = len(person_interacted_items_testset)

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id,
                                               items_to_ignore=get_items_interacted(person_id,
                                                                                    self.interactions_df_train),
                                               topn=1000, verbose=True)

        recall_metrics = self._calculate_recall(person_interacted_items_testset, person_recs_df, person_id)
        precision_metrics = self._calculate_precision(person_interacted_items_testset, person_recs_df, person_id)
        rr_metric = self._calculate_rr_user(person_interacted_items_testset, person_recs_df, person_id)

        person_metrics = {
            'recall': recall_metrics,
            'precision': precision_metrics,
            'rr': rr_metric,
        }

        return person_metrics

    def evaluate_model(self, model) -> tuple[pd.DataFrame, dict]:

        print(f'Evaluating {model.get_model_name()} recommendation model...')

        people_metrics_recall = []
        people_metrics_precision = []
        people_metrics_rr = []
        for idx, person_id in enumerate(list(self.interactions_df_test.index.unique().values)):

            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics_recall = person_metrics['recall']
            person_metrics_precision = person_metrics['precision']
            person_metrics_rr = person_metrics['rr']

            person_metrics_recall['_person_id'] = person_id
            people_metrics_recall.append(person_metrics_recall)

            person_metrics_precision['_person_id'] = person_id
            people_metrics_precision.append(person_metrics_precision)


            person_metrics_rr['_person_id'] = person_id
            people_metrics_rr.append(person_metrics_rr)
        print('%d users processed' % idx)

        detailed_results_recall_df = pd.DataFrame(people_metrics_recall) \
                            .sort_values('interacted_count', ascending=False)

        detailed_results_precision_df = pd.DataFrame(people_metrics_precision)

        detailed_results_rr_df = pd.DataFrame(people_metrics_rr)


        global_recall_at_5 = detailed_results_recall_df['hits@5_count'].sum() / float(detailed_results_recall_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_recall_df['hits@10_count'].sum() / float(detailed_results_recall_df['interacted_count'].sum())

        global_precision_at_5 = detailed_results_precision_df['hits@5_count'].sum() / float(detailed_results_precision_df.__len__() * 5)
        global_precision_at_10 = detailed_results_precision_df['hits@10_count'].sum() / float(detailed_results_precision_df.__len__() * 10)

        sum_rr = detailed_results_rr_df[detailed_results_rr_df['rr'] != -1]['rr'].sum()
        num_users = len(list(self.interactions_df_test.index.unique().values))
        mean_reciprocal_rank = (1 / int(num_users) ) * sum_rr

        global_metrics = {'modelName': [model.get_model_name()],
                          'recall@5': [global_recall_at_5],
                          'recall@10': [global_recall_at_10],
                          'precision@5': [global_precision_at_5],
                          'precision@10': [global_precision_at_10],
                          'mrr': [mean_reciprocal_rank]
                          }
        global_metrics_df = pd.DataFrame(global_metrics)
        dataframes = {'recall': detailed_results_recall_df,
                      'precision': detailed_results_precision_df,
                      'rr': detailed_results_rr_df
        }
        return global_metrics_df, dataframes

    def print_results(self, global_metrics, dataframes):
        print('\nGlobal metrics:\n%s' % global_metrics)
        for _, df in dataframes.items():
            print(df.head(50))

    def save_results(self, global_metrics: pd.DataFrame, dataframes: dict[str, pd.DataFrame], model_name, dataset_type):
        file_path = get_results_path(model_name, dataset_type)
        os.makedirs(file_path, exist_ok=True)
        global_metrics.to_csv(file_path + "global_metrics.csv")
        for key, df in dataframes.items():
            df.to_csv(file_path+key+".csv")


model_evaluator = ModelEvaluator(movie_ratings_indexed, interactions_train_df_indexed, interactions_test_df_indexed, movie_data)

# LLM content-based model

In [ ]:
!pip install -U sentence-transformers

In [ ]:
LLM_MODEL_NAME="BAAI/bge-m3"
# LLM_MODEL_NAME="dunzhang/stella_en_1.5B_v5"
# LLM_MODEL_NAME="dunzhang/stella_en_400M_v5"
# LLM_MODEL_NAME="Lajavaness/bilingual-embedding-large"


In [ ]:
# !pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [ ]:
# kwargs = {"torch_dtype": torch.float16}
kwargs={}

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# Load the model, optionally in float16 precision for faster inference
# model = SentenceTransformer("BAAI/bge-m3", model_kwargs={"torch_dtype": torch.float16})
model = SentenceTransformer(LLM_MODEL_NAME, trust_remote_code=True, model_kwargs=kwargs).cuda()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
def movie_imdb_stringified(movie_id: int) -> str:
    imdb_row = movie_imdb[movie_imdb['movieId'] == movie_id]
    if len(imdb_row) == 0:
        return ''
    imdb_row = imdb_row.iloc[0]
    return f"Cast: {imdb_row['Cast']}; Writers: {imdb_row['Writers']}; Producers: {imdb_row['Producers']}; Runtime: {imdb_row['Runtime']}; Box Office Budget: {imdb_row['Box Office Budget']}; Plot outline: {imdb_row['Plot outline']};"

In [ ]:
def stringify_movie(movies_df, movie_id) -> str:
    movie_doc = movies_df.loc[movie_id]['title']
    movie_genres = movies_df.loc[movie_id]['genres']
    movie_tags = movies_df.loc[movie_id]['tag']
    tags_str = ', '.join(movie_tags)
    genres_str = ", ".join(movie_genres)
    movie_doc = movie_doc + "; Genres: " + genres_str + "; Tags: " + tags_str
    return movie_doc

In [ ]:
# def stringify_movie(movies_df, movie_id) -> str:
#     movie_genres = movies_df.loc[movie_id]['genres']
#     movie_tags = movies_df.loc[movie_id]['tag']
#     tags_str = ', '.join(movie_tags)
#     genres_str = ", ".join(movie_genres)
#     movie_imdb_str = get_cast_imdb(movie_id)
#     movie_doc = "Genres: " + genres_str + "; Tags: " + tags_str + "; " + movie_imdb_str
#     return movie_doc

In [ ]:
def vectorize_movie(movies_df, movie_id):
    movie_doc = stringify_movie(movies_df, movie_id)
    movie_vec = model.encode([movie_doc])
    return movie_vec


### chroma

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.1 MB/s eta 0:00:

In [ ]:
from chromadb.api.types import (
    Documents,
    EmbeddingFunction,
    Embeddings
)


class NormalLLMEmbed(EmbeddingFunction[Documents]):
    def __init__(
            self
    ):
        """Initialize the embedding function."""

    def __call__(self, input: Documents) -> Embeddings:
        """Embed the input documents."""
        embeddings = [model.encode(doc).tolist() for doc in input]
        return embeddings

In [ ]:
class PromptedLLMEmbed(EmbeddingFunction[Documents]):
    def __init__(
            self,
            prompt: str
    ):
        """Initialize the embedding function."""
        self.prompt = prompt

    def __call__(self, input: Documents) -> Embeddings:
        """Embed the input documents."""
        embeddings = [model.encode(doc, prompt=self.prompt).tolist() for doc in input]
        return embeddings

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
# movie_emb = client.create_collection("movie_embeddings", embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(model_name=LLM_MODEL_NAME, trust_remote_code=True, device="cuda", model_kwargs=kwargs))
movie_emb = client.create_collection("movie_embeddings", embedding_function= NormalLLMEmbed())

In [ ]:
# client.delete_collection("movie_embeddings")

In [ ]:
documents = movie_data[CONTENT_COLUMN_NAME].apply(lambda x: stringify_movie(movie_data_indexed, x))
documents

,movieId
0,"Toy Story (1995); Genres: Adventure, Animation..."
1,"Jumanji (1995); Genres: Adventure, Children, F..."
2,"Grumpier Old Men (1995); Genres: Comedy, Roman..."
3,"Waiting to Exhale (1995); Genres: Comedy, Dram..."
4,Father of the Bride Part II (1995); Genres: Co...
...,...
9737,Black Butler: Book of the Atlantic (2017); Gen...
9738,No Game No Life: Zero (2017); Genres: Animatio...
9739,Flint (2017); Genres: Drama; Tags:
9740,Bungo Stray Dogs: Dead Apple (2018); Genres: A...


In [ ]:
document_ids = movie_data[CONTENT_COLUMN_NAME].apply(str)
document_ids.tolist()

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '34',
 '36',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '52',
 '53',
 '54',
 '55',
 '57',
 '58',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '85',
 '86',
 '87',
 '88',
 '89',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '110',
 '111',
 '112',
 '113',
 '116',
 '117',
 '118',
 '119',
 '121',
 '122',
 '123',
 '125',
 '126',
 '128',
 '129',
 '132',
 '135',
 '137',
 '140',
 '141',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '160',
 '161',
 '162',
 '163',
 

In [ ]:
len(document_ids)

9742

In [ ]:
movie_emb.upsert(
    ids= document_ids.tolist(),
    documents= documents.tolist(),
)


In [ ]:
encoding_instruction = "For given movies that user has already watched, separated by newline, given each movie's title, genres, tags and user rating, find relevant movies."
# encoding_instruction = "This is a user profile. It is given in a format where each movie the user has interacted with is on a new line. The movie is in the following format: Title:<TITLE>; Genres: <GENRE1, GENRE2 ...>; Tags: <TAG1, TAG2 ...>; Rating: <RATING\n. Find movies relevant to the ones provided: "
# encoding_instruction = "Encode each movie by focusing on its title, genres, tags, and user rating. Prioritize the extraction of thematic elements and genre characteristics, alongside user-specific preferences as reflected by their ratings. This encoding will inform a content-based recommendation system by capturing both the intrinsic attributes of the movies and the personalized preferences indicated by user ratings."
# encoding_instruction = "For a given user profile containing the user's preferred genres, tags and cast, find movies relevant to user."
prompt = f'<instruct>{encoding_instruction}\n<query>'


In [ ]:
# client.delete_collection("user_embeddings")

In [ ]:
user_emb = client.create_collection("user_embeddings", embedding_function=PromptedLLMEmbed(prompt))

### prompt variant

In [ ]:
movie_interactions_with_data = interactions_train_df.merge(movie_data, on=CONTENT_COLUMN_NAME)
movie_interactions_with_data.head()

,userId,movieId,rating,timestamp,title,genres,tag
0,140,4234,3.0,1012505945,"Tailor of Panama, The (2001)","[Drama, Thriller]",[]
1,156,2080,1.0,951113118,Lady and the Tramp (1955),"[Animation, Children, Comedy, Romance]",[Disney]
2,380,182639,4.0,1536874706,The Second Renaissance Part II (2003),"[Animation, Sci-Fi]",[]
3,495,5254,3.5,1458636268,Blade II (2002),"[Action, Horror, Thriller]","[Great movie, Wesley Snipes]"
4,610,69134,3.0,1493848172,Antichrist (2009),"[Drama, Fantasy]",[]


In [ ]:
movie_interactions_with_data['title'] = movie_interactions_with_data['title'].apply(lambda x: 'Title: '+ x)
movie_interactions_with_data['genres'] = movie_interactions_with_data['genres'].apply(lambda x: 'Genres: '+', '.join(x))
movie_interactions_with_data['tag'] = movie_interactions_with_data['tag'].apply(lambda x: 'Tags: '+', '.join(x))
movie_interactions_with_data['rating'] = movie_interactions_with_data['rating'].apply(lambda x: 'Rating: ' + str(x))
# movie_interactions_with_data['imdb'] = movie_interactions_with_data['movieId'].apply(lambda x: movie_imdb_stringified(x))

movie_interactions_with_data.head()

,userId,movieId,rating,timestamp,title,genres,tag
0,140,4234,Rating: 3.0,1012505945,"Title: Tailor of Panama, The (2001)","Genres: Drama, Thriller",Tags:
1,156,2080,Rating: 1.0,951113118,Title: Lady and the Tramp (1955),"Genres: Animation, Children, Comedy, Romance",Tags: Disney
2,380,182639,Rating: 4.0,1536874706,Title: The Second Renaissance Part II (2003),"Genres: Animation, Sci-Fi",Tags:
3,495,5254,Rating: 3.5,1458636268,Title: Blade II (2002),"Genres: Action, Horror, Thriller","Tags: Great movie, Wesley Snipes"
4,610,69134,Rating: 3.0,1493848172,Title: Antichrist (2009),"Genres: Drama, Fantasy",Tags:


In [ ]:
def get_cast_imdb(movie_id: int) -> str:
    imdb_row = movie_imdb[movie_imdb['movieId'] == movie_id]
    if len(imdb_row) == 0:
        return ''
    imdb_row = imdb_row.iloc[0]
    return str(imdb_row['Cast'])


In [ ]:
# movie_interactions_with_data['genres'] = movie_interactions_with_data['genres'].apply(lambda x:', '.join(x))
# movie_interactions_with_data['tag'] = movie_interactions_with_data['tag'].apply(lambda x:', '.join(x))
# movie_interactions_with_data['cast'] = movie_interactions_with_data['movieId'].apply(lambda x: get_cast_imdb(x))

# movie_interactions_with_data.head()

,userId,movieId,rating,timestamp,title,genres,tag,cast
0,140,4234,3.0,1012505945,"Tailor of Panama, The (2001)","Drama, Thriller",,"Pierce Brosnan, Geoffrey Rush, Jamie Lee Curti..."
1,156,2080,1.0,951113118,Lady and the Tramp (1955),"Animation, Children, Comedy, Romance",Disney,"Peggy Lee, Larry Roberts, Bill Baucom, Verna F..."
2,380,182639,4.0,1536874706,The Second Renaissance Part II (2003),"Animation, Sci-Fi",,"Julia Fletcher, Dane A. Davis, Debi Derryberry..."
3,495,5254,3.5,1458636268,Blade II (2002),"Action, Horror, Thriller","Great movie, Wesley Snipes","Wesley Snipes, Kris Kristofferson, Ron Perlman..."
4,610,69134,3.0,1493848172,Antichrist (2009),"Drama, Fantasy",,"Willem Dafoe, Charlotte Gainsbourg, Storm Ache..."


In [ ]:
movie_interactions_with_data['movie_str'] = movie_interactions_with_data[['title', 'genres', 'tag', 'rating']].agg('; '.join, axis=1)
# movie_interactions_with_data['movie_str'] = movie_interactions_with_data[['genres', 'tag', 'cast']].agg('; '.join, axis=1)
movie_interactions_with_data.head()

,userId,movieId,rating,timestamp,title,genres,tag,movie_str
0,140,4234,Rating: 3.0,1012505945,"Title: Tailor of Panama, The (2001)","Genres: Drama, Thriller",Tags:,"Title: Tailor of Panama, The (2001); Genres: D..."
1,156,2080,Rating: 1.0,951113118,Title: Lady and the Tramp (1955),"Genres: Animation, Children, Comedy, Romance",Tags: Disney,Title: Lady and the Tramp (1955); Genres: Anim...
2,380,182639,Rating: 4.0,1536874706,Title: The Second Renaissance Part II (2003),"Genres: Animation, Sci-Fi",Tags:,Title: The Second Renaissance Part II (2003); ...
3,495,5254,Rating: 3.5,1458636268,Title: Blade II (2002),"Genres: Action, Horror, Thriller","Tags: Great movie, Wesley Snipes","Title: Blade II (2002); Genres: Action, Horror..."
4,610,69134,Rating: 3.0,1493848172,Title: Antichrist (2009),"Genres: Drama, Fantasy",Tags:,"Title: Antichrist (2009); Genres: Drama, Fanta..."


In [ ]:
num_sampled = 50

In [ ]:
users_stringified = movie_interactions_with_data\
    .sort_values(by='timestamp', ascending=False)\
    .groupby(by=USER_COLUMN_NAME)\
    .head(num_sampled).groupby(by=USER_COLUMN_NAME)\
    .agg({'movie_str': '\n'.join})
users_stringified.head()

,movie_str
userId,
1,"Title: 20 Dates (1998); Genres: Comedy, Romanc..."
2,"Title: Town, The (2010); Genres: Crime, Drama,..."
3,"Title: You've Got Mail (1998); Genres: Comedy,..."
4,Title: Bridget Jones's Diary (2001); Genres: C...
5,Title: In the Line of Fire (1993); Genres: Act...


In [ ]:
# users_stringified = movie_interactions_with_data\
#     .sort_values(by='rating', ascending=False)\
#     .groupby(by=USER_COLUMN_NAME)\
#     .head(5)\
#     .groupby(by=USER_COLUMN_NAME)\
#     .agg({'genres': lambda x: ', '.join(filter(None, x)),'tag': lambda x: ', '.join(filter(None, x)), 'cast': lambda x: ', '.join(filter(None, x)) })

# users_stringified['user_str'] = "Genres: " + users_stringified['genres'] +"; Tags: "+users_stringified['tag']+"; Cast: "+users_stringified['cast']
# users_stringified.head()

,genres,tag,cast,user_str
userId,,,,
1,"Action, Adventure, Fantasy, Action, Crime, Dra...","transplants, mystery, twist ending, serial killer","Christopher Lambert, Roxanne Hart, Clancy Brow...","Genres: Action, Adventure, Fantasy, Action, Cr..."
2,"Action, Adventure, Sci-Fi, Thriller, Documenta...","beautiful, cinematography, visually appealing,...","Tom Hardy, Charlize Theron, Nicholas Hoult, Hu...","Genres: Action, Adventure, Sci-Fi, Thriller, D..."
3,"Action, Horror, Sci-Fi, Action, Adventure, Sci...",,"Ian McCulloch, Louise Marleau, Marino Masé, Si...","Genres: Action, Horror, Sci-Fi, Action, Advent..."
4,"Adventure, Fantasy, Musical, Crime, Drama, Thr...","drugs, based on a TV show, Australia, dance","David Bowie, Jennifer Connelly, Toby Froud, Sh...","Genres: Adventure, Fantasy, Musical, Crime, Dr..."
5,"Animation, Children, Drama, Fantasy, Musical, ...","Disney, writing, American Indians, Native Amer...","Roy Atwell, Stuart Buchanan, Adriana Caselotti...","Genres: Animation, Children, Drama, Fantasy, M..."


In [ ]:
user_id_list = users_stringified.index.astype(str).tolist()

### variant with mean vectors

In [ ]:
latest_n = 100

In [ ]:
user_movie_list = interactions_train_df.sort_values(by='timestamp', ascending=False).groupby(by=USER_COLUMN_NAME).agg({CONTENT_COLUMN_NAME: lambda x: list(x), 'rating': lambda x: list(x)})
user_movie_list['movieId'] = user_movie_list['movieId'].apply(lambda x: x[:latest_n])
user_movie_list['rating'] = user_movie_list['rating'].apply(lambda x: x[:latest_n])
user_movie_list.head()

,movieId,rating
userId,,
1,"[2478, 553, 1445, 157, 780, 1298, 3053, 3448, ...","[4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, ..."
2,"[80489, 122882, 1704, 89774, 80906, 68157, 109...","[4.5, 5.0, 4.5, 5.0, 5.0, 4.5, 3.0, 4.0, 4.0, ..."
3,"[3024, 7899, 5764, 1587, 26409, 2851, 70946, 5...","[4.5, 4.5, 4.5, 4.5, 4.5, 5.0, 5.0, 5.0, 5.0, ..."
4,"[4246, 4896, 4741, 4273, 4347, 4902, 4252, 476...","[4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 5.0, 3.0, ..."
5,"[247, 300, 474, 290, 475, 534, 232, 36, 410, 5...","[5.0, 3.0, 4.0, 5.0, 5.0, 3.0, 4.0, 4.0, 3.0, ..."


In [ ]:
from sklearn.preprocessing import normalize


In [ ]:
# returns single array of normalized, weighted average user embedding
def ids_to_embedding(ids: list, ratings: list):
    ids_str = [str(id) for id in ids]
    movies = movie_emb.get(ids_str, include=['embeddings'])
    embeddings = movies['embeddings']
    normalized_movie_embeddings = normalize(embeddings, axis=1)  # L2 normalize
    mean_user_embedding = np.mean(normalized_movie_embeddings * np.array(ratings)[:, np.newaxis], axis=0)
    # mean_user_embedding = np.mean(embeddings, axis=0)
    return mean_user_embedding

In [ ]:
embedding_list = user_movie_list.apply(lambda x: ids_to_embedding(x[CONTENT_COLUMN_NAME], x['rating']), axis=1).tolist()
embedding_list[:10]

[array([-0.02491527,  0.03901188, -0.11931839, ..., -0.04643117,
        -0.0379203 ,  0.00016088]),
 array([-0.06624984,  0.01443985, -0.07242631, ..., -0.0507566 ,
        -0.08951234,  0.0076736 ]),
 array([-0.04524171,  0.02870279, -0.13748634, ..., -0.06047079,
         0.00327658,  0.05336374]),
 array([-0.03874885,  0.0837678 , -0.07877526, ..., -0.02269583,
        -0.04064553, -0.0006416 ]),
 array([-0.06480074,  0.04675379, -0.09692731, ..., -0.05111133,
        -0.02386135,  0.00855367]),
 array([-0.024106  ,  0.04570985, -0.07105298, ..., -0.03381261,
        -0.01560013,  0.00958746]),
 array([-0.03774538,  0.05319159, -0.08596273, ..., -0.03563952,
        -0.02390424, -0.01503523]),
 array([-0.06304052,  0.06494665, -0.08517905, ..., -0.07408455,
        -0.0589463 ,  0.04196963]),
 array([-0.04392272,  0.05705021, -0.08498939, ..., -0.02660492,
        -0.03506951, -0.0204117 ]),
 array([-0.0571905 ,  0.07583146, -0.03454625, ..., -0.01458689,
        -0.05008547, -0.02

In [ ]:
user_id_list = [str(userId) for userId in user_movie_list.index.tolist()]
user_id_list[:10]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

### recommender

In [ ]:
user_emb.upsert(
    ids=user_id_list,
    documents=users_stringified['movie_str'].tolist()
)

In [ ]:
class LLMRecommender:

    MODEL_NAME = 'LLM recommender'


    def __init__(self, items_df=None):
        # self.popularity_df = popularity_df
        self.items_df = items_df


    def get_model_name(self):
        return self.MODEL_NAME


    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):

        user_embedding = user_emb.get([str(user_id)], include=['embeddings'])['embeddings']

        movie_ids = [movie for movie in movie_emb.get()['ids'] if int(movie) not in items_to_ignore] # get all ids NOT IN items to ignore list
        movie_embeddings = movie_emb.get(movie_ids, include=['embeddings'])['embeddings']

        similarities = model.similarity(user_embedding, movie_embeddings)
        similar_indices = similarities.argsort().flatten().tolist()[-topn:][::-1]

        similar_movies = [(int(movie_ids[i]), float(similarities[0][i])) for i in similar_indices]

        recommendations_df = pd.DataFrame(similar_movies, columns=[CONTENT_COLUMN_NAME, 'recStrength']).set_index(CONTENT_COLUMN_NAME)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left',
                                                          left_on = CONTENT_COLUMN_NAME,
                                                          right_on = CONTENT_COLUMN_NAME)


        return recommendations_df

llm_model = LLMRecommender(movie_data)

In [ ]:
llm_global_metrics, llm_results_dataframes = model_evaluator.evaluate_model(llm_model)
model_evaluator.print_results(llm_global_metrics, llm_results_dataframes)
model_evaluator.save_results(llm_global_metrics, llm_results_dataframes, llm_model.MODEL_NAME, DATASET_TYPE)

Evaluating LLM recommender recommendation model...
609 users processed

Global metrics:
{'modelName': 'LLM recommender', 'recall@5': 0.1671955573185244, 'recall@10': 0.25267750892502977, 'precision@5': 0.036065573770491806, 'precision@10': 0.031311475409836066, 'mrr': 0.11781324995199595}
     hits@5_count  hits@10_count  interacted_count  recall@5  recall@10  \
4              45             87               540  0.083333   0.161111   
29             47             88               496  0.094758   0.177419   
86             16             34               422  0.037915   0.080569   
40             52             85               373  0.139410   0.227882   
176            28             64               269  0.104089   0.237918   
32             38             56               261  0.145594   0.214559   
51             34             58               252  0.134921   0.230159   
133            55             87               244  0.225410   0.356557   
147             8             14   